<a href="https://colab.research.google.com/github/Esandu-Meth-Obadaarachchi/HCHO-level-Time-series-forecasting/blob/main/DE_2_NASA_DATASET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import requests
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=9fd38a700e2a1972b01c1811a9ab9981f6b387b13577d2105c0a2161da475615
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pyspark
import pyspark.sql  as pyspark_sql
import pyspark.sql.types as pyspark_types
import pyspark.sql.functions  as pyspark_functions
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col, regexp_replace, when
from pyspark.sql.types import StructType, StructField, DoubleType, StringType

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = pyspark_sql.SparkSession.builder.getOrCreate()

In [6]:
colombo_csv_path = '/content/drive/MyDrive/data engineering CW/colombo_df.csv'
deniyaya_matara_csv_path = '/content/drive/MyDrive/data engineering CW/deniyaya_matara_df.csv'
nuwara_eliya_csv_path = '/content/drive/MyDrive/data engineering CW/nuwara_eliya_df.csv'
bibile_monaragala_csv_path = '/content/drive/MyDrive/data engineering CW/bibile_monaragala_df.csv'
kurunegala_csv_path = '/content/drive/MyDrive/data engineering CW/kurunegala_df.csv'
jaffna_csv_path = '/content/drive/MyDrive/data engineering CW/jaffna_df.csv'
kandy_csv_path = '/content/drive/MyDrive/data engineering CW/kandy_df.csv'

In [7]:
colombo_df = spark.read.csv(colombo_csv_path,header = True)
deniyaya_matara_df = spark.read.csv(deniyaya_matara_csv_path,header = True)
nuwara_eliya_df = spark.read.csv(nuwara_eliya_csv_path,header = True)
bibile_monaragala_df = spark.read.csv(bibile_monaragala_csv_path,header = True)
jaffna_df = spark.read.csv(jaffna_csv_path,header = True)
kandy_df = spark.read.csv(kandy_csv_path ,header = True)
kurunegala_df=spark.read.csv(kurunegala_csv_path,header = True)

In [8]:
colombo_df.show()

+--------------+------------+----------+--------------------+
|      Location|Current Date| Next Date|        HCHO reading|
+--------------+------------+----------+--------------------+
|Colombo Proper|  2019-01-01|2019-01-02|1.969834395781014...|
|Colombo Proper|  2019-01-02|2019-01-03|2.625522171968594...|
|Colombo Proper|  2019-01-03|2019-01-04|9.852118897938794E-5|
|Colombo Proper|  2019-01-04|2019-01-05|2.099320518114242E-4|
|Colombo Proper|  2019-01-05|2019-01-06|1.785337298892930...|
|Colombo Proper|  2019-01-06|2019-01-07|1.082296700235670...|
|Colombo Proper|  2019-01-07|2019-01-08|3.926829280477309...|
|Colombo Proper|  2019-01-08|2019-01-09|9.153156350685351E-5|
|Colombo Proper|  2019-01-09|2019-01-10|1.205978992853015...|
|Colombo Proper|  2019-01-10|2019-01-11|1.297723562983258...|
|Colombo Proper|  2019-01-11|2019-01-12|2.239188166801278...|
|Colombo Proper|  2019-01-12|2019-01-13|1.569418094178759...|
|Colombo Proper|  2019-01-13|2019-01-14|1.569418094178759...|
|Colombo

In [9]:
colombo_data = spark.read.csv("/content/drive/MyDrive/data engineering CW/colombo_data.csv",header = True)

In [10]:
colombo_data.show()

+----+---+-----+-----+-----------+-----+------+------+-----+---------+-------+-------+-----------------+
|YEAR|DOY| QV2M| RH2M|PRECTOTCORR|  T2M|T2MDEW|T2MWET|   TS|T2M_RANGE|T2M_MAX|T2M_MIN|ALLSKY_SFC_LW_DWN|
+----+---+-----+-----+-----------+-----+------+------+-----+---------+-------+-------+-----------------+
|2019|  1|15.62|   80|       0.12| 24.8| 21.02| 22.91|25.29|     6.21|  28.44|  22.23|           407.15|
|2019|  2|15.14| 79.5|       0.02|24.39| 20.51| 22.45|25.01|     6.54|  28.02|  21.48|           401.33|
|2019|  3|14.77| 78.5|       0.01|24.17| 20.08| 22.13|24.73|     6.64|  27.94|   21.3|           395.24|
|2019|  4|14.47|77.62|          0|24.04| 19.74| 21.89|24.57|     7.66|  28.36|  20.71|           384.66|
|2019|  5|16.05| 81.5|       0.03|24.87| 21.36| 23.12|25.33|     8.26|  29.23|  20.97|           399.29|
|2019|  6|14.89|77.62|          0|24.56| 20.19| 22.37|25.08|     6.88|   28.4|  21.54|           387.12|
|2019|  7|15.87|79.75|       0.02|25.04| 21.16|  23.1|2

In [12]:
jaffna_data = spark.read.csv("/content/drive/MyDrive/data engineering CW/jaffna_data.csv",header = True)
kandy_data = spark.read.csv("/content/drive/MyDrive/data engineering CW/kandy_data.csv",header = True)
kurunagala_data = spark.read.csv("/content/drive/MyDrive/data engineering CW/kurunagala_data.csv",header = True)
nuwara_eliya_data = spark.read.csv("/content/drive/MyDrive/data engineering CW/nuwara_eliya_data.csv",header = True)
deniyaya_matara_data = spark.read.csv("/content/drive/MyDrive/data engineering CW/matara_data.csv",header = True)
monaragala_data = spark.read.csv("/content/drive/MyDrive/data engineering CW/monaragala_data.csv",header = True)

In [16]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
from datetime import datetime, timedelta

from datetime import datetime, timedelta

# Define a function to convert Day of Year to date
def doy_to_date(year, doy):
    # Construct the date using year and day of year
    date = datetime(year, 1, 1) + timedelta(days=int(doy) - 1)
    return date.strftime('%Y-%m-%d')

# Example usage
year = 2019
doy = 1
date = doy_to_date(year, doy)
print(date)  # Output: '2019-01-01'

2019-01-01


In [19]:
# Assuming your DataFrame is named df
print(jaffna_data.dtypes)

[('YEAR', 'string'), ('MO', 'string'), ('DY', 'string'), ('QV2M', 'string'), ('RH2M', 'string'), ('PRECTOTCORR', 'string'), ('T2M', 'string'), ('T2MDEW', 'string'), ('T2MWET', 'string'), ('TS', 'string'), ('T2M_RANGE', 'string'), ('T2M_MAX', 'string'), ('T2M_MIN', 'string')]


In [21]:
colombo_data['YEAR'] = colombo_data['YEAR'].astype(int)
colombo_data['DOY'] = colombo_data['DOY'].astype(int)

In [23]:
import pandas as pd

# Apply the function to create a new 'Date' column
colombo_data['Date'] = colombo_data.apply(lambda row: doy_to_date(row['YEAR'], row['DOY']), axis=1)

# Drop the 'YEAR' and 'DOY' columns if needed
colombo_data.drop(columns=['YEAR', 'DOY'], inplace=True)

# Now your 'Date' column will contain dates in the format 'YYYY-MM-DD'
print(colombo_data.head())

    QV2M   RH2M PRECTOTCORR    T2M T2MDEW T2MWET     TS T2M_RANGE T2M_MAX  \
0  15.62     80        0.12   24.8  21.02  22.91  25.29      6.21   28.44   
1  15.14   79.5        0.02  24.39  20.51  22.45  25.01      6.54   28.02   
2  14.77   78.5        0.01  24.17  20.08  22.13  24.73      6.64   27.94   
3  14.47  77.62           0  24.04  19.74  21.89  24.57      7.66   28.36   
4  16.05   81.5        0.03  24.87  21.36  23.12  25.33      8.26   29.23   

  T2M_MIN ALLSKY_SFC_LW_DWN        Date  
0   22.23            407.15  2019-01-01  
1   21.48            401.33  2019-01-02  
2    21.3            395.24  2019-01-03  
3   20.71            384.66  2019-01-04  
4   20.97            399.29  2019-01-05  


In [27]:
colombo_data = spark.createDataFrame(colombo_data)

In [28]:
colombo_data.show()

+-----+-----+-----------+-----+------+------+-----+---------+-------+-------+-----------------+----------+
| QV2M| RH2M|PRECTOTCORR|  T2M|T2MDEW|T2MWET|   TS|T2M_RANGE|T2M_MAX|T2M_MIN|ALLSKY_SFC_LW_DWN|      Date|
+-----+-----+-----------+-----+------+------+-----+---------+-------+-------+-----------------+----------+
|15.62|   80|       0.12| 24.8| 21.02| 22.91|25.29|     6.21|  28.44|  22.23|           407.15|2019-01-01|
|15.14| 79.5|       0.02|24.39| 20.51| 22.45|25.01|     6.54|  28.02|  21.48|           401.33|2019-01-02|
|14.77| 78.5|       0.01|24.17| 20.08| 22.13|24.73|     6.64|  27.94|   21.3|           395.24|2019-01-03|
|14.47|77.62|          0|24.04| 19.74| 21.89|24.57|     7.66|  28.36|  20.71|           384.66|2019-01-04|
|16.05| 81.5|       0.03|24.87| 21.36| 23.12|25.33|     8.26|  29.23|  20.97|           399.29|2019-01-05|
|14.89|77.62|          0|24.56| 20.19| 22.37|25.08|     6.88|   28.4|  21.54|           387.12|2019-01-06|
|15.87|79.75|       0.02|25.04| 21.16

In [30]:
 # Assuming colombo_data is your PySpark DataFrame and you want to drop the 'YEAR' and 'DOY' columns
colombo_data = colombo_data.drop('T2MDEW', 'T2MWET', 'T2M_RANGE', 'T2M_MAX', 'T2M_MIN', 'ALLSKY_SFC_LW_DWN')


In [31]:
colombo_data.show()

+-----+-----+-----------+-----+-----+----------+
| QV2M| RH2M|PRECTOTCORR|  T2M|   TS|      Date|
+-----+-----+-----------+-----+-----+----------+
|15.62|   80|       0.12| 24.8|25.29|2019-01-01|
|15.14| 79.5|       0.02|24.39|25.01|2019-01-02|
|14.77| 78.5|       0.01|24.17|24.73|2019-01-03|
|14.47|77.62|          0|24.04|24.57|2019-01-04|
|16.05| 81.5|       0.03|24.87|25.33|2019-01-05|
|14.89|77.62|          0|24.56|25.08|2019-01-06|
|15.87|79.75|       0.02|25.04|25.63|2019-01-07|
|16.66|81.25|       0.01|25.58|25.96|2019-01-08|
|16.36|80.69|       0.04|25.37|25.81|2019-01-09|
|16.72|81.19|       0.23|25.61|26.08|2019-01-10|
|16.91|   83|        6.5|25.37|25.96|2019-01-11|
| 17.4|84.12|       2.04|25.65|25.94|2019-01-12|
|17.76|86.25|       5.47|25.56|25.79|2019-01-13|
|17.76|85.06|       4.44| 25.8|25.94|2019-01-14|
|17.21|83.56|       1.18|25.59|25.83|2019-01-15|
|17.21|83.81|       3.96|25.53|25.73|2019-01-16|
|15.32|78.56|       0.15|24.78|25.09|2019-01-17|
| 14.4|   77|       

In [32]:
colombo_data = colombo_data.withColumnRenamed('QV2M', 'Specific Humidity')
colombo_data = colombo_data.withColumnRenamed('RH2M', 'Relative Humidity')

In [33]:
colombo_data.show()

+-----------------+-----------------+-----------+-----+-----+----------+
|Specific Humidity|Relative Humidity|PRECTOTCORR|  T2M|   TS|      Date|
+-----------------+-----------------+-----------+-----+-----+----------+
|            15.62|               80|       0.12| 24.8|25.29|2019-01-01|
|            15.14|             79.5|       0.02|24.39|25.01|2019-01-02|
|            14.77|             78.5|       0.01|24.17|24.73|2019-01-03|
|            14.47|            77.62|          0|24.04|24.57|2019-01-04|
|            16.05|             81.5|       0.03|24.87|25.33|2019-01-05|
|            14.89|            77.62|          0|24.56|25.08|2019-01-06|
|            15.87|            79.75|       0.02|25.04|25.63|2019-01-07|
|            16.66|            81.25|       0.01|25.58|25.96|2019-01-08|
|            16.36|            80.69|       0.04|25.37|25.81|2019-01-09|
|            16.72|            81.19|       0.23|25.61|26.08|2019-01-10|
|            16.91|               83|        6.5|25